In [3]:
import pandas as pd
import numpy as np

In [60]:
# run
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [ ]:
credits.sample(5)
movies.sample(5)

In [61]:
# run
movies = movies.merge(credits,on = "title")

In [13]:
movies.shape

(4809, 23)

In [ ]:
# remove unnecessary columns
# genre, id, keywords, title, overview, cast, crew, 
movies.info()

In [62]:
# run
movies = movies[["id",  "title", "overview", 'genres',  "keywords",  "cast", "crew"]]

In [63]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [64]:
# run
# duplicate and missing value
movies.isnull().sum()
movies.dropna(inplace=True)
movies.duplicated().sum()

np.int64(0)

In [66]:
# run
# '[
    # {"id": 28, "name": "Action"}, 
    # {"id": 12, "name": "Adventure"}, 
    # {"id": 14, "name": "Fantasy"}, 
    # {"id": 878, "name": "Science Fiction"}
# ]'
import ast
movies.iloc[0].genres
def convert(string):
    ouput = []
    for i in ast.literal_eval(string):
        ouput.append(i['name'])
    return ouput

def convert_cast(string):
    ouput = []
    c = 0
    for i in ast.literal_eval(string):
        if c < 3:
            ouput.append(i['name'])
            c+=1
        else:
            break
    return ouput

def fetch_director(string):
    ouput = []
    for i in ast.literal_eval(string):
        if i['job'] == "Director":
            ouput.append(i['name'])
            break
    return ouput

In [67]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert_cast)
movies['crew'] = movies['crew'].apply(fetch_director)
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [69]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [71]:
movies['tags'] = movies['keywords'] + movies['cast'] + movies['genres'] + movies['crew'] 

In [83]:
df = movies[['id','title','tags']]
df['tags'] = df['tags'].apply(lambda x: " ".join(x))
df['tags'] = df['tags'].apply(lambda x: x.lower())

C:\Users\admin\AppData\Local\Temp\ipykernel_20056\2224977180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: " ".join(x))
C:\Users\admin\AppData\Local\Temp\ipykernel_20056\2224977180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: x.lower())


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500,stop_words='english')

In [11]:
import pandas as pd
df = pd.read_csv("cleaned_movies.csv")
# df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
# df = df.drop('paltu',axis=1)
# df.to_csv("cleaned_file.csv", index=False)

In [12]:
df = df.dropna(subset=['tags'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4793 entries, 0 to 4802
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4793 non-null   int64 
 1   title   4793 non-null   object
 2   tags    4793 non-null   object
dtypes: int64(1), object(2)
memory usage: 149.8+ KB


In [14]:
from nltk.stem.porter import PorterStemmer
ks = PorterStemmer()

def stem(text):
    ouput = []
    for i in text.split():
        ouput.append(ks.stem(i))
    return " ".join(ouput)

df['tags'] = df['tags'].apply(stem)

In [15]:
vectors = cv.fit_transform(df['tags']).toarray()

# array(['18thcentury', '1930s', '1940s', ..., 'zombieapocalypse', 'zoo',
    #    'zooeydeschanel'], shape=(2500,), dtype=object)
cv.get_feature_names_out()

array(['18thcenturi', '1930', '1940', ..., 'zombieapocalyps', 'zoo',
       'zooeydeschanel'], shape=(2500,), dtype=object)

In [16]:
# we find the cosine similarity of two vectors,
# theta between two vectors

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
print(similarity)

[[1.         0.14301939 0.12856487 ... 0.07537784 0.         0.        ]
 [0.14301939 1.         0.13483997 ... 0.         0.         0.        ]
 [0.12856487 0.13483997 1.         ... 0.         0.         0.        ]
 ...
 [0.07537784 0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [24]:
def recommend(movie):
    index = df[df['title'] == movie].index[0]
    distances = similarity[index]
    # sort based on the second index 
    movies_list =  sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:6]
    for i in movies_list:
        print(df.iloc[i[0]].title)
    

In [34]:
recommend("Batman Begins")

The Dark Knight
The Dark Knight Rises
Amidst the Devil's Wings
Blood Done Sign My Name
Close Range


In [ ]:
import pickle
pickle.dump(df,open("movies.pkl","wb"))

In [17]:
pickle.dump(similarity,open("similarity.pkl","wb"))


In [18]:
pickle.dump(df.to_dict(),open("movies_dict.pkl","wb"))